In [1]:
Query="What are all the education qualification of shyam?"

In [3]:
from dotenv import load_dotenv

load_dotenv()


True

In [4]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

def process_pdf_batch(pdf_file):
    loader = PyPDFLoader(pdf_file)
    pages = loader.load_and_split()
    character_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0
        )
    pages = character_splitter.split_documents(pages)
    return pages

In [6]:
d="resume"
chunks=process_pdf_batch(fr"data\{d}.pdf")

In [8]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=chunks, 
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [14]:
retriever.invoke(Query)

[Document(page_content='Shyam Sundar\n5,kavimani steet, Pankajam colony, 3rd cross street, Madurai, India - 625009\n♂phone+91-9080765574 /envel⌢pemailshyamsundar.2022@gmail.com /linkedinshyamsundar007 /githubShyam-Sundar-7♂laptopPortfolio\nProjects\nLLM-Powered Coupon Recommender /github/video |Python, Streamlit, Langchain, OpenAI November 2023\n•Developed a QA system for e-commerce with personalized coupon recommendations using OpenAI’s LLMs.\n•Streamlined user interactions through a Streamlit interface and Langchain for real-world scenario simulations.\n•Incorporated FAISS for refined recommendation processes.\nPeopleCare Insurance Prediction /github |Python, Jupyter, Azure Cloud, Flask, Docker October 2023\n•Expanded PeopleCare into vehicle insurance with a predictive model for effective customer targeting.\n•Thorough analysis of customer behavior and data cleaning for accurate predictive modeling.\n•Achieved 80% prediction accuracy using LightGBM.', metadata={'page': 0, 'source': '

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.load import dumps, loads

def generate_queries():
        
    # Multi Query: Different Perspectives
    template = """You are an AI language model assistant. Your task is to generate Four 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Original question: {question}"""
    prompt_perspectives = ChatPromptTemplate.from_template(template)


    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() 
        | (lambda x: x.split("\n"))
    )
    return generate_querie 


def _get_docs(m):
    docs=[]
    docs.extend(m['original'])
    for i in m['new']:
        docs.extend(i)
    return docs



In [10]:
from langchain_core.runnables import RunnableParallel
original_question= retriever
retrieval_chain =  generate_queries() | retriever.map()
map_chain = RunnableParallel(original=original_question,new=retrieval_chain) | _get_docs

In [11]:
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
embedding_function = SentenceTransformerEmbeddingFunction()

c:\Users\shyams\Downloads\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\shyams\Downloads\RAG\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shyams\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In orde

In [13]:
import numpy as np
import tqdm

def project_embeddings(embeddings, umap_transform):
    umap_embeddings = np.empty((len(embeddings),2))
    for i, embedding in enumerate(tqdm(embeddings)): 
        umap_embeddings[i] = umap_transform.transform([embedding])
    return umap_embeddings

In [16]:
import chromadb

from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import numpy as np
from pypdf import PdfReader
from tqdm import tqdm


def _read_pdf(filename):
    reader = PdfReader(filename)
    
    pdf_texts = [p.extract_text().strip() for p in reader.pages]

    # Filter the empty strings
    pdf_texts = [text for text in pdf_texts if text]
    return pdf_texts


def _chunk_texts(texts):
    character_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=1000,
        chunk_overlap=0
    )
    character_split_texts = character_splitter.split_text('\n\n'.join(texts))

    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

    token_split_texts = []
    for text in character_split_texts:
        token_split_texts += token_splitter.split_text(text)

    return token_split_texts

def load_chroma(filename, collection_name, embedding_function):
    texts = _read_pdf(filename)
    chunks = _chunk_texts(texts)

    chroma_cliet = chromadb.Client()
    chroma_collection = chroma_cliet.create_collection(name=collection_name, embedding_function=embedding_function)

    ids = [str(i) for i in range(len(chunks))]

    chroma_collection.add(ids=ids, documents=chunks)

    return chroma_collection

In [15]:
chroma_collection = load_chroma(filename='data\resume.pdf', collection_name='resume', embedding_function=embedding_function)
chroma_collection.count()

NameError: name 'load_chroma' is not defined

In [ ]:
embeddings = chroma_collection.get(include=['embeddings'])['embeddings']
umap_transform = umap.UMAP(random_state=0, transform_seed=0).fit(embeddings)
projected_dataset_embeddings = project_embeddings(embeddings, umap_transform)